```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Perpendicular random fields

This notebook is used to test the generation of perpendicular random fields produced on the GPU. Hipp hurra!

## Set environment

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation, rc

import pycuda.driver as cuda
import os
import sys
from importlib import reload
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import OceanStateNoise, IPythonMagic

In [ ]:
%cuda_context_handler gpu_ctx
gpu_stream = cuda.Stream()

In [ ]:
def imshow2(xi, nu, interpolation="None", title=None):
    fig, axs = plt.subplots(1,2, figsize=(8,4))
    
    xi_im = axs[0].imshow(xi, interpolation=interpolation, origin='lower')
    axs[0].set_title("xi")
    plt.colorbar(xi_im, ax=axs[0])
    
   
    nu_im = axs[1].imshow(nu, interpolation=interpolation, origin='lower')
    axs[1].set_title("nu")
    plt.colorbar(nu_im, ax=axs[1])

    if title is not None:
        plt.suptitle(title)
    plt.tight_layout()
 

# Testing 


In [ ]:
if 'newnoise' in globals():
    newnoise.cleanUp()
reload(OceanStateNoise)

nx, ny = 210, 210
dx, dy = 100.0, 100.0
interpolation_factor = 1

noise = OceanStateNoise.OceanStateNoise(gpu_ctx, gpu_stream, \
                                        nx, ny, dx, dy, \
                                        Common.BoundaryConditions(2,2,2,2),
                                        False,
                                        interpolation_factor=interpolation_factor)

print("----- Initial reduction buffer ----------")
reduction_buffer = noise.getReductionBuffer()
print("xi_norm, nu_norm, xi_nu_dot: ", (reduction_buffer[0,0], reduction_buffer[0,1], reduction_buffer[0,2]))


print("------- Indepentend xi, nu sim N(0,I) ---------")
noise.generateNormalDistribution()
noise.generateNormalDistributionPerpendicular()
noise.findDoubleNormAndDot()
reduction_buffer_host = noise.getReductionBuffer()

xi = noise.getRandomNumbers()
nu = noise.getPerpendicularRandomNumbers()

print("xi.shape: ", xi.shape)
print("nu.shape: ", nu.shape)

imshow2(xi, nu, title="Independent xi, nu sim N(0,I)")

xiTnu = np.sum(xi*nu)
xiTxi = np.sum(xi*xi)
nuTnu = np.sum(nu*nu)
print("CPU xiTnu: ", xiTnu)
print("CPU nuTnu: ", nuTnu)
print("CPU xiTxi: ", xiTxi)

print("Reduction buffer: ")
print("\txi_norm, nu_norm, xi_nu_dot: ", 
      (reduction_buffer_host[0,0], reduction_buffer_host[0,1], reduction_buffer_host[0,2]))


print("----------- Make perpendicular --------------")
noise._makePerpendicular()
xi_2 = noise.getRandomNumbers()
nu_2 = noise.getPerpendicularRandomNumbers()

print("is xi unchanged? sum(norm(xi - xi_2)): ", np.sum((xi - xi_2)*(xi - xi_2)))
print("is |nu| unchanged? norm(nu) - norm(nu_2): ", np.sum(nu_2*nu_2) - nuTnu)

imshow2(xi_2, nu_2, title="Perpendicular xi, nu sim N(0,I)")

noise.findDoubleNormAndDot()
reduction_buffer_host_2 = noise.getReductionBuffer()
print("xi_2_norm, nu_2_norm, xi_2_nu_2_dot: ",
      (reduction_buffer_host_2[0,0], reduction_buffer_host_2[0,1], reduction_buffer_host_2[0,2]))


print("dot product of final downloaded buffers: ", np.sum(xi_2*nu_2))